In [1]:
from getpass import getpass
import pandas as pd
import requests
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta, SU,MO
import datetime

import json

from google.colab.data_table import DataTable
DataTable.max_columns = 36


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open('WB').sheet1
 
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)
 
import pandas as pd
labels = [
          'product','retail_price_withdisc_rub',	'commission_percent',	'delivery_rub', 'storage'
          ]
idx = [x[0] for x in rows]
df = pd.DataFrame.from_records(rows, columns=labels, index=idx)
wb = df.iloc[1: , :]
cols = wb.columns.drop(['product'])
wb[cols] = wb[cols].apply(pd.to_numeric, errors='coerce')
wb

[['product', 'retail_price_withdisc_rub', 'commission_percent', 'delivery_rub', 'storage'], ['ProWaxWhite', '641', '0.12', '75', '0.14'], ['ProWaxWhiteReNewed', '618', '0.12', '75', '0.14'], ['ProWaxSingleWhite', '613', '0.12', '75', '0.14'], ['BeautyCreameCushion01', '241', '0.1', '63.25', '0.05'], ['BeautyCreameCushion02', '241', '0.1', '63.25', '0.05'], ['EnoughCollagen21', '400', '0.1', '45', '0.05'], ['EnoughCollagen13', '400', '0.1', '45', '0.05'], ['EnoughCollagen23', '400', '0.1', '45', '0.05'], ['sifat 1000', '600', '0.12', '75', '0.05']]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,product,retail_price_withdisc_rub,commission_percent,delivery_rub,storage
ProWaxWhite,ProWaxWhite,641,0.12,75.00,0.14
ProWaxWhiteReNewed,ProWaxWhiteReNewed,618,0.12,75.00,0.14
ProWaxSingleWhite,ProWaxSingleWhite,613,0.12,75.00,0.14
BeautyCreameCushion01,BeautyCreameCushion01,241,0.10,63.25,0.05
BeautyCreameCushion02,BeautyCreameCushion02,241,0.10,63.25,0.05
EnoughCollagen21,EnoughCollagen21,400,0.10,45.00,0.05
EnoughCollagen13,EnoughCollagen13,400,0.10,45.00,0.05
EnoughCollagen23,EnoughCollagen23,400,0.10,45.00,0.05
sifat 1000,sifat 1000,600,0.12,75.00,0.05


In [ ]:
def get_orders(date_from, date_to=""):
  query = f'https://suppliers-stats.wildberries.ru/api/v1/supplier/orders?key={key}&dateFrom={date_from}&dateTo={date_to}'
  if  date_to == "":
    query = f'https://suppliers-stats.wildberries.ru/api/v1/supplier/orders?key={key}&dateFrom={date_from}'
  response = requests.get(query)
  if response.status_code != 200:
    print(f"Status code: {response.status_code}")
    return pd.DataFrame()
  
  payload = response.json()
  return pd.DataFrame.from_records(payload)

def get_sales(date_from, date_to=""):
  query = f'https://suppliers-stats.wildberries.ru/api/v1/supplier/sales?key={key}&dateFrom={date_from}&dateTo={date_to}'
  if date_to == "":
    query = f'https://suppliers-stats.wildberries.ru/api/v1/supplier/sales?key={key}&dateFrom={date_from}'
    
  response = requests.get(query)
  if response.status_code != 200:
    print(f"Status code: {response.status_code}")
    return pd.DataFrame()
  
  payload = response.json()
  return pd.DataFrame.from_records(payload)

In [ ]:
key = getpass('Enter the key: ')

Enter the key: ··········


# Week

In [ ]:
last_monday = today + relativedelta(weekday=MO(-1))
week_orders_full = get_orders(last_monday, today)
week_orders = week_orders_full[week_orders_full['isCancel'] == False].groupby(['supplierArticle'])['supplierArticle'].count().to_frame()
returns = week_orders_full[week_orders_full['isCancel'] == True].groupby(['supplierArticle'])['supplierArticle'].count().to_frame()
week_sales_full = get_sales(last_monday, today)
week_sales = week_sales_full.groupby(['supplierArticle'])['supplierArticle'].count().to_frame()

In [ ]:
week_orders

In [ ]:
returns

In [ ]:
week_sales

In [ ]:
week_sales_full =  pd.merge(week_sales_full, wb, how='left', left_on=['supplierArticle'], right_on = ['product'])
week_sales_full['transfer_to_account'] = week_sales_full.forPay - week_sales_full.forPay * week_sales_full.commission_percent - week_sales_full.delivery_rub

In [ ]:
print(week_sales_full.transfer_to_account.sum())

In [ ]:
week_returns = week_orders_full[week_orders_full['isCancel'] == True]
week_returns

# Last week

In [ ]:
today = date.today()
prev_monday = today + relativedelta(weekday=MO(-2))
last_sunday =  today + relativedelta(weekday=SU(-1))
last_week_orders_full = get_orders(prev_monday, last_sunday)
last_week_orders = last_week_orders_full[last_week_orders_full['isCancel'] == False].groupby(['supplierArticle'])['supplierArticle'].count().to_frame()
last_week_returns = last_week_orders_full[last_week_orders_full['isCancel'] == True].groupby(['supplierArticle'])['supplierArticle'].count().to_frame()
last_week_sales_full = get_sales(prev_monday, last_sunday)
last_week_sales = last_week_sales_full.groupby(['supplierArticle'])['supplierArticle'].count().to_frame()

https://suppliers-stats.wildberries.ru/api/v1/supplier/sales?key=YTY5YmUxYWQtYWIyNi00YmU4LThjODctNjA2YmYzNTA1ZDk5&dateFrom=2022-08-29&dateTo=2022-09-04


In [ ]:
last_week_sales_full.date = pd.to_datetime(last_week_sales_full.date)

sun_day = datetime.datetime.strptime(last_sunday, '%Y/%m/%d')
last_week_sales_full = last_week_sales_full[last_week_sales_full['date'] < sun_day]
last_week_sales_full

,date,lastChangeDate,supplierArticle,techSize,barcode,totalPrice,discountPercent,isSupply,isRealization,promoCodeDiscount,...,finishedPrice,priceWithDisc,nmId,subject,category,brand,IsStorno,gNumber,sticker,srid
0,2022-08-29 07:25:14,2022-08-29T07:27:06,BeautyCreameCushion01,0,2000018782002,4841,95,False,True,0,...,300.00,300.00,70382904,Кушоны,Красота,,0,3505791305713572885,,18673e8cb16f41a9ace5f060642e339c
1,2022-08-29 11:16:04,2022-08-29T11:17:31,BeautyCreameCushion02,0,2031828066624,4821,95,False,True,0,...,299.00,299.05,80637492,Кушоны,Красота,,0,98035068011690583521,,ba6207b87a4a4a98863cf4aabd9444d5
2,2022-08-29 12:45:24,2022-08-29T12:45:24,ProWaxWhite,0,2021803974006,12821,95,False,True,0,...,651.70,715.05,65718762,Воскоплавы,Бытовая техника,Pro Wax100,0,278229757418280485,,c40efb5f43f54df5b670c37fa616772e
3,2022-08-29 13:24:20,2022-08-29T13:24:27,BeautyCreameCushion02,0,2031828066624,4821,95,False,True,0,...,269.00,269.05,80637492,Кушоны,Красота,,0,94717901911568680238,,8d9a2959e4e24f438fc1a5a962347e28
4,2022-08-30 09:44:39,2022-08-30T09:45:35,EnoughCollagen13,0,2031166258620,7201,95,False,True,0,...,400.00,400.00,79196781,Кремы,Красота,ENOUGH,0,7835138088659033137,,df862c0ad0574030a7b3f37615109a55
5,2022-08-30 10:02:24,2022-08-30T10:02:24,ProWaxWhite,0,2021803974006,12821,95,False,True,0,...,651.70,715.05,65718762,Воскоплавы,Бытовая техника,Pro Wax100,0,97741894778897972909,,75395c59fec746ff9bbfe2d5ad584f6c
6,2022-08-30 16:38:55,2022-08-30T16:40:24,BeautyCreameCushion01,0,2000018782002,4841,95,False,True,0,...,270.00,270.05,70382904,Кушоны,Красота,,0,92004100696137903326,,018e9f15fc3343a1af8aff98c79e57d1
7,2022-08-30 17:27:27,2022-08-30T17:27:33,ProWaxWhite,0,2021803974006,12821,95,False,True,0,...,651.70,715.05,65718762,Воскоплавы,Бытовая техника,Pro Wax100,0,194013186291177859,,6d995503bd5a4989aefc28da73c0957e
8,2022-08-30 17:46:53,2022-08-30T17:48:44,ProWaxWhite,0,2021803974006,12821,95,False,True,0,...,651.70,715.05,65718762,Воскоплавы,Бытовая техника,Pro Wax100,0,98166756926455362681,,e9145d09a74c4c76912a5026ce316fdc
9,2022-08-30 19:46:37,2022-08-30T19:47:16,BeautyCreameCushion02,0,2031828066624,4821,95,False,True,0,...,269.00,269.05,80637492,Кушоны,Красота,,0,891530995517319915,,999189caad6a41e684cf745b5733af68
